In [2]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle


In [3]:
# Đường dẫn tới thư mục dự án
path = '/thigiacmaytinh'
# Hiển thị tất cả thư mục hoặc file
print(os.listdir(path))

['.git', '01.ipynb', '02.ipynb', '03.ipynb', '04.ipynb', '05.ipynb', '06.ipynb', '07.ipynb', '08.ipynb', '09.ipynb', '10.ipynb', '11.ipynb', '12.ipynb', '13.ipynb', '14.ipynb', '15.ipynb', '16.ipynb', '17.ipynb', '18.ipynb', '19.ipynb', '20.ipynb', '21.ipynb', '22.ipynb', '23.ipynb', '24.ipynb', '25.ipynb', '26.ipynb', '31.ipynb', '32.ipynb', 'data', 'data_face_features.pickle', 'FaceImage', 'hinhanh', 'imgtool.py', 'models', 'PP', 'video', '__pycache__']


In [4]:
# Face detection model
faceDetectionModel = path + "/models/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# Mô tả kiến trúc
faceDetectionProto = path + "/models/deploy.proto.txt"

# Mô hình mô tả khuôn mặt
faceDescriptor = path + "/models/openface.nn4.small2.v1.t7"

# Sử dụng OpenCV DNN đọc mô hìn nhận diện khuôn mặt
dectectorModel = cv2.dnn.readNetFromCaffe(faceDetectionProto, faceDetectionModel)

# Đọc mô hình mô tả khuôn mặt từ file Torch
descriptorModel = cv2.dnn.readNetFromTorch(faceDescriptor)

In [5]:
# Danh sách hình ảnh chuẩn bị
faces = path + '/FaceImage'
os.listdir(faces)

['151901205_BuiKhanhBinh',
 '162000186_NguyenVanHuong',
 '162000223_ChuNguyenTienThanh',
 '162000309_LeHoangQuan',
 '162000339_HuynhAnhDuy',
 '162000651_NguyễnXuânNam',
 '162000670_nguyenmanhthi',
 '162000947_TranThanhDat',
 '162000989_NguyenThanhSang',
 '162001371_leminhtien',
 '162001391_TranTien',
 '1711031462_VanHuuDuc',
 '172100041_HoangHuuDien',
 '172100043_NguyenHuuHa',
 '172100048_NguyenPhamAnhKhoa',
 '172100062_Trần_Nguyễn_Ngọc_Quý',
 '172100095_NguyenTrungNghia',
 '172100103_tranquangthuan',
 '172100123_NguyenTrongAn',
 '172100124_BuiMinhHieu',
 '172100177_DangNguyenThienAn',
 '172100187_LeAnhQuan',
 '172100191_LeMinhKhoi',
 '172100201_NguyenThanhDat',
 '172100207_DuHieuBao',
 '172100251_NguỵenHaiDuy',
 '172100257_LuongThanhPhong',
 '172100264_NgoDangDuy',
 '172100280_NguyenHoangQuan',
 '172100304_LiuGiaLuan',
 '172100309_TranCongSang',
 '172100321_NguyenTrongNghia',
 '172100326_DangVuTrangNguyen',
 '172100337_HaDuyPhuong',
 '172100357_LeQuangPhu',
 '172100358_DoanVuBinhDuong

In [6]:
# Trích xuất đặc trưng cho một ảnh
quang = path + '/FaceImage/1721030545_NguyenHoangTuanAnh/image_20.jpg'
img1 = cv2.imread(quang)

# Copy cái ảnh
image = img1.copy()

# Lấy chiều cao và chiều rộng của ảnh
h, w = image.shape[:2]

# Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
imgBlob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 177, 123), swapRB=False, crop=False)

# Thiết lập đầu vào cho mô hình
dectectorModel.setInput(imgBlob)

# Thực hiện việc nhận diện khuôn mặt
detections = dectectorModel.forward()

# Kiểm tra xem có khuôn mặt nào hay không?
if (len(detections) > 0):
    # Chọn khuôn mặt có độ tin cậy cao nhất (confidence) cao nhất
    i = np.argmax(detections[0, 0, :, 2])  # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
    confidence = detections[0, 0, i, 2]

    # Kiểm tra độ tin cậy có lớn hơn 0.5
    if (confidence > 0.5):
        # Tính toán hộp bao quanh khuôn mặt
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')

        # Trích xuất vùng ảnh chứa khuôn mặt ra
        roi = image[startY:endY, startX:endX]
        cv2.imshow('Detected Face', roi)
        cv2.waitKey(0)  # Đợi người dùng nhấn phím bất kỳ để đóng cửa sổ
        cv2.destroyAllWindows()

        # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
        faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96, 96), (0, 0, 0), swapRB=True, crop=True)

        # Thiết lập đầu vào cho mô hình
        descriptorModel.setInput(faceBlob)

        # Thực hiện việc trích xuất đặc trưng
        vectors = descriptorModel.forward()

        # Print vectors
        print(vectors)


[[ 0.09935124  0.10685824 -0.12078106  0.01040769  0.09279075  0.15504114
   0.03306207 -0.01996637 -0.07365546  0.04796164  0.10352544  0.09059916
   0.11966831 -0.14109756 -0.05129287 -0.05484948  0.00820175  0.12226451
  -0.15128456  0.20945393  0.01069102 -0.04086575  0.08296976 -0.02031151
  -0.06059804 -0.10527267 -0.09051228 -0.07942857 -0.06838731  0.0455832
  -0.0362959   0.01427094 -0.04358507  0.10674074 -0.02118528 -0.00433792
  -0.0192175   0.03943756 -0.03654089 -0.00212133 -0.01544046 -0.05909093
  -0.00886094  0.10526714 -0.2263596  -0.1355312   0.08224359  0.0622915
  -0.05212526  0.24773061  0.1258099  -0.04941595 -0.25360844  0.05114198
  -0.01473654 -0.07500169 -0.08630021 -0.00605618  0.00901395 -0.00869827
  -0.07425056 -0.04150065  0.13416442 -0.16309173  0.06726245 -0.23200607
   0.00644841  0.0046408  -0.13964571  0.06437384  0.07528705  0.10247701
   0.07092144  0.07689993 -0.11877426 -0.04149755  0.02058363  0.04572079
  -0.01028509  0.03614715 -0.04421205 -0

In [7]:
# Tạo ra một hàm để áp dụng cho nhiều ảnh
def myDetect(image_path):
    # Trích xuất đặc trưng cho một ảnh
    img1 = cv2.imread(image_path)

    # Copy cái ảnh
    image = img1.copy()

    # Lấy chiều cao và chiều rộng của ảnh
    h, w = image.shape[:2]

    # Chuẩn bị dữ liệu đầu vào cho mô hình nhận diện khuôn mặt
    imgBlob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 177, 123), swapRB=False, crop=False)

    # Thiết lập đầu vào cho mô hình
    dectectorModel.setInput(imgBlob)

    # Thực hiện việc nhận diện khuôn mặt
    detections = dectectorModel.forward()

    # Kiểm tra xem có khuôn mặt nào hay không?
    if (len(detections) > 0):
        # Chọn khuôn mặt có độ tin cậy (confidence) cao nhất
        i = np.argmax(detections[0, 0, :, 2])  # Giá trị thứ 2 trong 7 giá trị cuối cùng => confidence
        confidence = detections[0, 0, i, 2]

        # Kiểm tra độ tin cậy có lớn hơn 0.5
        if (confidence > 0.5):
            # Tính toán hộp bao quanh khuôn mặt
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            # Trích xuất vùng ảnh chứa khuôn mặt ra
            roi = image[startY:endY, startX:endX]

            # Chuẩn bị dữ liệu đầu vào cho mô hình trích xuất đặc trưng
            faceBlob = cv2.dnn.blobFromImage(roi, 1/255, (96, 96), (0, 0, 0), swapRB=True, crop=True)

            # Thiết lập đầu vào cho mô hình
            descriptorModel.setInput(faceBlob)

            # Thực hiện việc trích xuất đặc trưng
            vectors = descriptorModel.forward()
            return vectors


In [8]:
# Apply for all images
# Khởi tạo một dictionary
data = dict(data=[], label=[])

myDir = '/thigiacmaytinh/FaceImage'
folders = os.listdir(myDir)

# Duyệt qua từng thư mục
for folder in folders:
  path = myDir + "/"+folder
  print(path)

  files =  os.listdir(path)
  for fileName in files:
    try:
        # Gọi hàm trích xuất đặc trưng
        vector = myDetect(path+'/'+fileName)

        # Kiểm tra việc trích xuất đặc trưng thành công:
        if vector is not None:
          #Thêm vector đặc trưng vào trong từ điển với nhãn là tên của thư mục
          data['data'].append(vector)
          data['label'].append(folder)
          print('Trích xuất đặc trưng thành công')
    except:
      # Bỏ qua nếu gặp lỗi
      pass

/thigiacmaytinh/FaceImage/151901205_BuiKhanhBinh
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
/thigiacmaytinh/FaceImage/162000186_NguyenVanHuong
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Trích xuất đặc trưng thành công
Tríc

In [9]:
# Tạo một series từ dictionary
labelSeries = pd.Series(data['label'])
# Đếm số lần xuất hiện của mỗi nhãn
labelCounts = labelSeries.value_counts()
print(labelCounts)

1721030566_LeNguyenHoang     150
1721030848_LeQuocKy          135
1721031129_TranAnhHao        130
172100257_LuongThanhPhong    121
1721030367_LePhuongLinh      119
                            ... 
1721030489_VuNgocKyDuyen      10
1721031660_LeQuangQuocDat     10
1721030497_HoThienSinh         9
172100103_tranquangthuan       8
1721030802_NguyenNhatTien      5
Name: count, Length: 87, dtype: int64


In [10]:
# Save data -> file
print('/thigiacmaytinh/')
fileName = '/thigiacmaytinh/data_face_features.pickle'
pickle.dump(data, open(fileName, mode='wb'))

/thigiacmaytinh/
